In [1]:
#! pip install nglview
# /home/shenwanxiang/anaconda3/envs/tf2/bin/jupyter-nbextension enable nglview --py --sys-prefix
#!pip install --upgrade MDAnalysis
#conda install -c omnia pdbfixer

In [7]:
#所用蛋白为1stp
from pdbfixer import PDBFixer
from simtk.openmm.app.pdbfile import PDBFile
from simtk.openmm.app import ForceField,  Simulation, PME, HBonds, PDBReporter, StateDataReporter
from simtk.openmm.app import forcefield
from simtk.openmm import app, LangevinIntegrator
from simtk.unit import nanometer, kelvin, picosecond, picoseconds
from sys import stdout

import nglview as nv
import os

In [1]:
import sys, time
import openmmforcefields
from simtk import unit, openmm
from simtk.openmm import app, Platform, LangevinIntegrator
from simtk.openmm.app import PDBFile, Simulation, Modeller, PDBReporter, StateDataReporter, DCDReporter
from rdkit import Chem
from openforcefield.topology import Molecule

RDKit WARNING: [21:31:03] Enabling RDKit 2019.09.2 jupyter extensions


ModuleNotFoundError: No module named 'openforcefield'

In [ ]:
openmmforcefields.

In [9]:
simtk.__version__

AttributeError: module 'simtk' has no attribute '__version__'

In [11]:
! pip install openmmforcefields==0.7.4

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement openmmforcefields==0.7.4
ERROR: No matching distribution found for openmmforcefields==0.7.4
You should consider upgrading via the '/home/shenwanxiang/anaconda3/envs/tf2/bin/python -m pip install --upgrade pip' command.


In [3]:
def fix_pdb(pdb_id):
    path = os.getcwd()
    if len(pdb_id) != 4:
        print("Creating PDBFixer...")
        fixer = PDBFixer(pdb_id)
        print("Finding missing residues...")
        fixer.findMissingResidues()

        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                print("ok")
                del fixer.missingResidues[key]

        print("Finding nonstandard residues...")
        fixer.findNonstandardResidues()
        print("Replacing nonstandard residues...")
        fixer.replaceNonstandardResidues()
        print("Removing heterogens...")
        fixer.removeHeterogens(keepWater=True)

        print("Finding missing atoms...")
        fixer.findMissingAtoms()
        print("Adding missing atoms...")
        fixer.addMissingAtoms()
        print("Adding missing hydrogens...")
        fixer.addMissingHydrogens(7)
        print("Writing PDB file...")

        PDBFile.writeFile(
            fixer.topology,
            fixer.positions,
            open(os.path.join(path, "%s_fixed_pH_%s.pdb" % (pdb_id.split('.')[0], 7)),
                 "w"),
            keepIds=True)
        return "%s_fixed_pH_%s.pdb" % (pdb_id.split('.')[0], 7)

In [4]:
ls

00_MD_protein_only_test.ipynb  1stp.pdb   output.pdb  SVM/
1stp_fixed_pH_7.pdb            input.pdb  readme.md


In [5]:
view = nv.show_structure_file('./1stp.pdb')
view

NGLWidget()

In [6]:
new_pdb_name = fix_pdb('1stp.pdb')

Creating PDBFixer...
Finding missing residues...
ok
ok
Finding nonstandard residues...
Replacing nonstandard residues...
Removing heterogens...
Finding missing atoms...
Adding missing atoms...
Adding missing hydrogens...
Writing PDB file...


In [19]:
#加载pdb文件，同样openmm也支持PDBx/mmCIF格式
pdb = app.PDBFile('1stp_fixed_pH_7.pdb')
pdb

In [20]:
nv.show_structure_file('./1stp_fixed_pH_7.pdb')

NGLWidget()

In [21]:
#确定使用的力场，Amber14：amber14-all.xml；TIP3P-FB water model：amber14/tip3pfb.xml
#使用XML files 来定义标准力场
#可用力场从：http://docs.openmm.org/latest/userguide/application.html#force-fields 进行查看
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
#将力场与分子拓扑文件相组合来创建一个对此模拟系统的完整的数学描述
#一些options：
# 将粒子网格Ewald用于长距离静电相互作用（nonbondedMethod = PME）
# 设置1nm截止值用于直接空间相互作用（nonbondedCutoff = 1 *纳米）
# 并约束所有涉及氢原子的键的长度（约束条件 = HBonds）。
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,nonbondedCutoff=1*nanometer, constraints=HBonds)
# 朗之万动力学 ( Langevin Dynamics ) ，模拟温度（300 K），摩擦系数（1 ps-1）和步长（0.004 ps）
# 还可以使用许多其他集成方法。 例如，如果要在恒定能量而不是恒定温度下模拟系统，则可以使用VerletIntegrator。
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
# 将topology，system，integrator进行组合，创建一个Simulation对象，并将其分配给一个名为Simulation的变量。
#模拟对象管理运行模拟所涉及的所有过程
simulation = Simulation(pdb.topology, system, integrator)
#指定了用于模拟的初始原子位置。
simulation.context.setPositions(pdb.positions)
#进行局部能量最小化，建议在模拟开始前进行，减少应力
simulation.minimizeEnergy()
#创建一个“报告程序”以在模拟过程中输出结果， PDBReporter将结构写入PDB文件。
#指定输出文件为output.pdb，并应每1000步写入一个结构。
simulation.reporters.append(PDBReporter('output.pdb', 1000))
#在模拟运行时获取常规状态报告，可以监视其进度
#添加另一个报告程序，以每1000个时间步长打印一些基本信息：当前步长索引，系统的势能和温度。
#将stdout（而不用引号）指定为输出文件，这意味着将结果在终端显示。
#也可以给文件名（用引号引起来），将信息写到文件中。
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,potentialEnergy=True, temperature=True))
#开始模拟
simulation.step(10000)
#result

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-10679.407383238133,303.5574211544042
2000,-10596.262851988133,304.8758687867087
3000,-10736.559726988133,304.4199591183918
4000,-10375.942539488133,301.2545330287171
5000,-10787.462070738133,303.6179204317578
6000,-10909.770664488133,296.62547342912495
7000,-10979.544101988133,309.15981442302234
8000,-10863.188633238133,293.6229570200309
9000,-10748.731601988133,301.89200142243476
10000,-11123.860508238133,303.71234820619463


In [22]:
pdb.topology

<Topology; 2 chains, 205 residues, 1996 atoms, 1934 bonds>

In [ ]:
# from simtk.openmm.app import *
# from simtk.openmm import *
# from simtk.unit import *
# from sys import stdout

# pdb = PDBFile('input.pdb')
# forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
# system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
#         nonbondedCutoff=1*nanometer, constraints=HBonds)
# integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
# simulation = Simulation(pdb.topology, system, integrator)
# simulation.context.setPositions(pdb.positions)
# simulation.minimizeEnergy()
# simulation.reporters.append(PDBReporter('output.pdb', 1000))
# simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
#         potentialEnergy=True, temperature=True))
# simulation.step(10000)